In [10]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Let's import first the stock returns, weights, and other variables you generated in the previous notebook as well as variance-covariance VaR calculation:


In [11]:
stocks_returns = pd.read_csv('stocks_returns.csv')
stocks_returns = stocks_returns.set_index('Date')
weights = np.load('weights.npy')
total_asset = 1e6
conf_level = 0.95
t = 5

In [12]:
stocks_returns_mean = stocks_returns.mean().mean()
cov_var = stocks_returns.cov()
port_std = np.sqrt(weights.T.dot(cov_var).dot(weights))

def VaR_parametric(total_asset, conf_level, port_std, t):
    alpha = norm.ppf(conf_level, stocks_returns_mean)
    VaR_param = np.round((total_asset * port_std * np.sqrt(t) * alpha), 2)
    return VaR_param

VaR_param = VaR_parametric(total_asset, conf_level, port_std, t)
VaR_param

56073.26

# Expected Shortfall

Unlike VaR, Expected Shortfall focuses on the tail of the distribution. More elaboratively, Expected Shortfall enable us to take into account the unexpected risks in the market. However, it does not mean that Expected Shortfall and VaR are two entirely different concepts. Rather, they are related, that is, it is possible to express Expected Shortfall using VaR.


 Expected Shortfall formula suggests that it is nothing but a probability weighted average of $(1-\alpha)%$ of losses. 

\begin{equation}
\text{ES}_{\alpha}=\frac{1}{1-\alpha}\int^1_{\alpha} q_u du
\end{equation}

where q denotes quantile of the loss distribution. So, all we need to do is to calculate the VAR based on the quantile (q) and then calculate the integral, which will be done using `expect` function below.

Let's now calculate the expected shortfall using the above-provided formula using Python:

In [13]:
def ES(initial_investment, conf_level, port_std, stocks_returns_mean):
    alpha = norm.ppf(conf_level, stocks_returns_mean, port_std)
    VaR_param = VaR_parametric(total_asset, conf_level, port_std, t)
    ES_result = (1 / (1 - conf_level)) * norm.expect(lambda x:  VaR_param, 
                                                                  lb = 1 - conf_level)
    return np.round(ES_result, 2)

In [14]:
ES(total_asset, conf_level, port_std, stocks_returns_mean)

538371.92

It turns out $538371.92 is the average amount that is lost over a 5-day period, assuming that the loss is greater than the 95th percentile of the loss distribution.